### Bài toán

Các khó khăn gặp phải khi xây dựng mô hình phân loại văn bản tiếng Việt:

1. Việc tách từ tiếng Việt khá là phức tạp do có nhiều từ và cụm từ ghép lại với nhau, nhiều từ không có trong từ điển.
2. Việc tìm được bộ từ điển tương đối đầy đủ khá khó khăn do tiếng Việt có nhiều từ ngữ cả trong lĩnh vực chuyên ngành và nhiều từ ngữ không có trong từ điển. Hiện tại mới chỉ tìm được một số ít từ phổ biến có trong các từ điển mở.
3. Việc triển khai mô hình phân loại văn bản tiếng Việt khá là khó khăn do không có nhiều tài liệu tham khảo, đặc biệt là các tài liệu về xử lý ngôn ngữ tự nhiên tiếng Việt.
4. Việc cài đặt môi trường lập trình cho Tensorflow cũng như pytorch gặp nhiều khó khăn khi cài đặt trên Windows, đặc biệt là khi sử dụng GPU.
5. Việc chuẩn hoá văn bản tiếng Việt gặp rất nhiều thách thức về:
    - Cách mã hoá văn bản Unicode
    - Cách bỏ dấu tiếng Việt
    - Xử lý sai chính tả
    - Xử lý từ viết tắt
    - Xử lý các từ ngữ không có trong từ điển
7. Việc tách từ cũng gặp rất nhiều bất lợi
    - Từ điển càng lớn thì việc tách từ càng chậm, đã thử nghiệm và tách từ mất 30p cho tập train.
    - Tokenization phải đào tạo mô hình để thực hiện mới đem lại hiệu quả tốt. Đã thử nghiệm với các thư viện như pyvi, underthesea, vncorenlp, ... nhưng kết quả không tốt. Chậm quá
6. Hiện chưa có bộ StopWords tiếng Việt tương đối đầy đủ, nên việc xử lý StopWords cũng gặp nhiều khó khăn do đa phần là tự tạo không theo một tiêu chuẩn nào cả.

- Input: một đoạn văn bản
- Output: một trong các nhãn sau: 'XH':'Chinh tri Xa hoi', 
            'DS':'Doi song', 
            'KH':'Khoa hoc', 
            'KD':'Kinh doanh', 
            'PL':'Phap luat', 
            'SK':'Suc khoe', 
            'TG':'The gioi', 
            'TT':'The thao', 
            'VH':'Van hoa', 
            'VT':'Vi tinh',



### Thu thập dữ liệu


Dữ liệu được lấy dừ https://github.com/duyvuleo/VNTC/tree/master/Data/10Topics/Ver1.1 với thông tin như sau:

***Train***
|Topic |	Topic ID |	#files |
|--|--|--|
| Chinh tri Xa hoi	| XH |	5219 |
| Doi song	| DS |		3159 |
| Khoa hoc	| KH |		1820 |
| Kinh doanh	| KD |		2552 |
| Phap luat	| PL |		3868 |
| Suc khoe	| SK |		3384 |
| The gioi	| TG |		2898 |
| The thao	| TT |		5298 |
| Van hoa 	| VH |		3080 |
| Vi tinh		| VT |		2481 |

Total				33759

***Test***
|Topic |	Topic ID |	#files |
|--|--|--|
| Chinh tri Xa hoi	| XH |	7567 |
| Doi song	| DS |		2036 |
| Khoa hoc	| KH |		2096 |
| Kinh doanh	| KD |		5276 |
| Phap luat	| PL |		3788 |
| Suc khoe	| SK |		5417 |
| The gioi	| TG |		6716 |
| The thao	| TT |		6667 |
| Van hoa		| VH |		6250 |
| Vi tinh		| VT |		4560 |

Total				50373

Tên file được đặt cào từ:

+ DS_VNE_(...) : VnExpress news agency (http://vnexpress.net/)
+ DS_TT_(...):  Youth news agency (http://tuoitre.vn/)
+ DS_TN_(...): Thanh Nien news agency (http://thanhnien.vn/)
+ DS_NLD_(...): Nguoi Lao Dong news agency (http://nld.com.vn/)

File zip chứa toàn bộ file, tên mỗi file là Nhãn_Báo_ (STT).txt VD: XH_NLD_ (3675).txt

In [36]:
import zipfile

def extract_data_from_zip(zip_file_path, length = 50000):
    result = []
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()  # Lấy danh sách tên các file trong zip

        for file_name in file_list:
            if file_name.endswith('.txt'):  # Chỉ xử lý các file có đuôi .txt
                with zip_ref.open(file_name) as file:
                    data = file.read().decode('utf-16-le')  # Nó được mã hoá bằng utf-16-le
                    # nội dung file có dạng \ufeff<content>, đôi khi có dấu cách ở đầu và cuối
                    content = data[1:].strip()
                    words = content.split(' ')
                    content = ' '.join(words[:min(length, len(words))])
                    label = file_name.split('/')[2].split('_')[:2]
                    result.append([content, label])
    return result

In [37]:
train_data = extract_data_from_zip('data/Train_Full.zip')
len(train_data)

33759

In [38]:
test_data = extract_data_from_zip('data/Test_Full.zip')
len(test_data)

50373

In [4]:
labels = [x[0] for _, x in train_data]
labels = set(labels)
labels

{'DS', 'KD', 'KH', 'PL', 'SK', 'TG', 'TT', 'VH', 'VT', 'XH'}

In [5]:
label_dict={'XH':'Chinh tri Xa hoi', 
            'DS':'Doi song', 
            'KH':'Khoa hoc', 
            'KD':'Kinh doanh', 
            'PL':'Phap luat', 
            'SK':'Suc khoe', 
            'TG':'The gioi', 
            'TT':'The thao', 
            'VH':'Van hoa', 
            'VT':'Vi tinh', }
print(*[label_dict[x] for x in labels], sep = ', ')

Phap luat, Suc khoe, Khoa hoc, Vi tinh, Kinh doanh, The gioi, The thao, Van hoa, Chinh tri Xa hoi, Doi song


### Tiền xử lý dữ liệu

#### Xoá HTML

Dữ liệu được thu thập từ các website đôi khi vẫn còn sót lại các đoạn mã HTML. Các mã HTML code này là rác, chẳng những không có tác dụng cho việc phân loại mà còn làm kết quả phân loại văn bản bị kém đi. Do đó, cần phải loại bỏ các đoạn mã HTML này.

In [6]:
import re
def remove_html(txt):
    return re.sub(r'<[^>]*>', ' ', txt)

txt = "<p class=\"par\">This is an example</p>nè"
remove_html(txt)

' This is an example nè'

#### Chuẩn hoá Tiếng Việt

- **Chuẩn hoá Unicode**: Hiện nay, có 2 loại mã Unicode được sử dụng phổ biến, Unicode tổ hợp và Unicode dựng sẵn. Hướng xử lý: Đưa về 1 chuẩn Unicode dựng sẵn (thằng này phổ biến hơn)
- **Chuẩn hoán cách bỏ dấu**: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý

In [7]:
import re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

dicchar = loaddicchar()

def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)
    
dict_map = {"òa": "oà", "Òa": "Oà", "ÒA": "OÀ", "óa": "oá", "Óa": "Oá", "ÓA": "OÁ", "ỏa": "oả", "Ỏa": "Oả", "ỎA": "OẢ", "õa": "oã", "Õa": "Oã", "ÕA": "OÃ", "ọa": "oạ", "Ọa": "Oạ", "ỌA": "OẠ", "òe": "oè", "Òe": "Oè", "ÒE": "OÈ", "óe": "oé", "Óe": "Oé", "ÓE": "OÉ", "ỏe": "oẻ", "Ỏe": "Oẻ", "ỎE": "OẺ", "õe": "oẽ", "Õe": "Oẽ", "ÕE": "OẼ", "ọe": "oẹ", "Ọe": "Oẹ", "ỌE": "OẸ", "ùy": "uỳ", "Ùy": "Uỳ", "ÙY": "UỲ", "úy": "uý", "Úy": "Uý", "ÚY": "UÝ", "ủy": "uỷ", "Ủy": "Uỷ", "ỦY": "UỶ", "ũy": "uỹ", "Ũy": "Uỹ", "ŨY": "UỸ", "ụy": "uỵ", "Ụy": "Uỵ", "ỤY": "UỴ",}

def replace_all(text, dict_map):
    for i, j in dict_map.items():
        text = text.replace(i, j)
    return text
def vietnameseTextNormalizer(sentence):
    import unicodedata    
    return replace_all(unicodedata.normalize('NFC', convert_unicode(sentence)), dict_map)

str_utf8 = 'Anh Hòa, đang làm gì chị Thúy vậy, ăn qụyt phải không?' # Unicode (dựng sẵn - dấu theo ký tự)
str_utf8_2 = 'Anh Hoà, đang làm gì chị Thuý vậy, ăn quỵt phải không?' # Unicode (dựng sẵn - dấu theo ký tự)
str_com = 'Anh Hòa, đang làm gì chị Thúy vậy, ăn qụyt phải không?'  # Unicode composite (tổ hợp - dấu riêng)
str_1252 = 'Anh Hòa, đang làm gì chị Thúy vậy, ăn qụyt phải không?' # Windows-1252 = Latin-1
print(str_utf8 == str_utf8_2, str_utf8 == str_com, str_utf8 == str_1252, str_com == str_1252)
str_utf8 = vietnameseTextNormalizer(str_utf8)
str_utf8 = vietnameseTextNormalizer(str_utf8_2)
str_com = vietnameseTextNormalizer(str_com)
str_1252 = vietnameseTextNormalizer(str_1252)
print(str_utf8 == str_utf8_2, str_utf8 == str_com, str_utf8 == str_1252, str_com == str_1252)

False False False True
True True True True


#### Tách từ

File từ điển các từ và cụm từ được tổng hợp từ các nguồn sau:
- Viet74K.txt: https://github.com/undertheseanlp/underthesea/tree/main/underthesea/corpus/data
- words.txt: https://github.com/undertheseanlp/underthesea/tree/main/datasets/DI_Vietnamese-UVD/corpus/dictionary
- vi-vocab: https://github.com/vncorenlp/VnCoreNLP/tree/master/models/wordsegmenter
- Thư mục Words - Danh mục từ của wordnet: https://github.com/zeloru/vietnamese-wordnet/tree/master

Sau đó được xử lý để tạo thành 1 file từ điển duy nhất tên là [dic3.txt](data/dic3.txt) nhờ code từ: [Create_Data.ipynb](Create_Data.ipynb)

In [52]:
from collections import defaultdict
import re

def syllablize(sentence): # Tách âm tiết cho một câu tiếng Việt
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

# Tải từ trong vi-vocab.txt
with open('data/dic3.txt', encoding='utf8') as f:
    vocab = f.read().split('\n')
# Xây dựng từ điển vocabs theo độ dài từ
vocabs = defaultdict(dict) # dùng list chậm hơn so với dict khoảng 25 lần
for word in vocab:
    vocabs[len(word.split())][word.lower()] = True

print('Số lượng từ ghép và cụm từ trong vocab:', len(vocab))
print('Số bộ vocab phân theo độ dài:', len(vocabs))

def longest_matching(sentence, vocabs):
  words = syllablize(sentence) # tách âm tiết cho câu
  result = []
  i = len(words)-1 # index của từ hiện tại
  while i > -1: 
    word = '' 
    # tìm kiếm trong từ điển theo chiều dài của từ ưu tiên từ dài trước
    for j in range(max(i-8,0), i+1):
      ls_word = words[j:i+1]
      word = ' '.join(ls_word)
      # xem thử có trong từ điển không
      if word.lower() in vocabs.get(i-j+1, []):
        i = j
        break
    result.append(word)
    i-=1
  return result[::-1] # return the final list

def tokenize_sentences(sentence):
    return ' '.join([x.replace(' ','_') for x in longest_matching(sentence, vocabs)])

tokenize_sentences('nhưng sự thực hiện vẫn còn chưa phù hợp')

Số lượng từ ghép và cụm từ trong vocab: 112343
Số bộ vocab phân theo độ dài: 20


'nhưng sự_thực_hiện vẫn_còn chưa phù_hợp'

#### Hoàn thiện tiền xử lý

In [9]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = vietnameseTextNormalizer(document)
    # tách từ
    document = tokenize_sentences(document)
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

text = text_preprocess('<p class=\"par\">Têu đề bài báo:</p>Có vẻ như Anh Hòa, đang làm gì đó với chị Thúy vậy, có thể là ăn qụyt có phải không?')
text

'têu đề bài_báo có_vẻ_như anh hoà đang làm_gì đó với chị thuý vậy có_thể_là ăn_quỵt có_phải không'

#### Loại bỏ stopword

In [10]:
with open('data/stopwords-nlp-vi.txt', encoding='utf8') as f:
    stopword = f.read().replace(' ','_').split('\n')

stopword = set(stopword)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

text = remove_stopwords(text)
text

'têu đề bài_báo có_vẻ_như hoà thuý có_thể_là ăn_quỵt'

### Xây dựng mô hình phân loại văn bản

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

if torch.cuda.is_available():
    print("PyTorch is using GPU.")
    device = torch.device("cuda")  # Chọn GPU làm thiết bị tính toán
    print(f"Current GPU device: {torch.cuda.current_device()}")
    print(f"Number of available GPUs: {torch.cuda.device_count()}")
else:
    print("PyTorch is using CPU.")
    device = torch.device("cpu")  # Chọn CPU là thiết bị tính toán


PyTorch is using GPU.
Current GPU device: 0
Number of available GPUs: 1


Trong mô hình phân loại văn bản thì `vocab_size`, `embedding_dim`, `hidden_size`, và `num_classes` là các thông số quan trọng cần được xác định trước khi xây dựng mô hình:

1. `vocab_size` là kích thước của từ điển (vocabulary) trong mô hình phân loại văn bản. Nó đại diện cho số lượng từ duy nhất có trong tập dữ liệu huấn luyện. Khi xây dựng mô hình, mỗi từ sẽ được biểu diễn bằng một chỉ số số nguyên từ 0 đến `vocab_size - 1`. `vocab_size` có thể được tính toán tự động bằng cách đếm từ.

Để tính toán `vocab_size`, bạn có thể xây dựng từ điển từ các từ trong tập dữ liệu huấn luyện và đếm số lượng từ duy nhất. Một cách đơn giản để làm điều này là sử dụng một bộ đếm từ (word counter) để đếm số lần xuất hiện của mỗi từ trong tập dữ liệu và sau đó lấy độ dài của từ điển để có `vocab_size`.

2. `embedding_dim`: Đây là số chiều của không gian nhúng (embedding space) trong mô hình. Trong quá trình huấn luyện, các từ trong từ điển sẽ được biểu diễn bằng các vectơ có kích thước `embedding_dim`. Số chiều này cần được chọn sao cho đủ lớn để mô hình có thể học được các đặc trưng quan trọng của văn bản, nhưng cũng không quá lớn để tránh tăng quá nhiều tham số và tốn thời gian huấn luyện.Thông thường, kích thước không gian nhúng từ 100 đến 300 chiều đã được sử dụng hiệu quả trong nhiều nhiệm vụ phân loại văn bản.

3. `hidden_size`: Đây là số lượng đơn vị ẩn trong mạng LSTM (Long Short-Term Memory) hoặc các mạng RNN (Recurrent Neural Network) khác. `hidden_size` ảnh hưởng đến khả năng mô hình học các mẫu dữ liệu phức tạp. Nếu `hidden_size` lớn, mô hình có khả năng học các mẫu phức tạp hơn, nhưng đồng thời tăng cường độ phức tạp của mô hình và thời gian huấn luyện. Giá trị thông thường cho `hidden_size` trong mạng LSTM hoặc RNN là 100, 200 hoặc 300. Tuy nhiên, nếu tập dữ liệu lớn hoặc bài toán phân loại phức tạp hơn, có thể cần tăng giá trị này để mô hình có khả năng học mẫu phức tạp hơn.

4. `num_classes`: Đây là số lượng lớp trong bài toán phân loại. Đối với bài toán phân loại văn bản, `num_classes` sẽ là số lượng nhãn khác nhau mà chúng ta muốn mô hình phân loại các văn bản vào. Ví dụ, nếu ta có 3 nhãn: "positive", "negative", và "neutral", thì `num_classes` sẽ là 3.

Việc lựa chọn các giá trị tối ưu cho `embedding_dim`, `hidden_size`, và `num_classes` phụ thuộc vào bài toán cụ thể và dữ liệu đang làm việc. Thông thường, các giá trị này được chọn dựa trên kinh nghiệm thực tế và thử nghiệm. Trong quá trình huấn luyện, bạn có thể điều chỉnh các giá trị này và theo dõi hiệu suất của mô hình để tìm ra các giá trị phù hợp nhất.

In [40]:
def getPreprocessed_text(text):
    text = text_preprocess(text)
    text = remove_stopwords(text)
    return text

In [13]:
# Lớp tập dữ liệu tùy chỉnh
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __getitem__(self, index):
        text, label = self.data[index]
        return torch.tensor(getPreprocessed_text(text)), label
    
    def __len__(self):
        return len(self.data)
    

In [53]:
from collections import Counter
from tqdm import tqdm

# Tính toán từ điển và vocab_size
word_counter = Counter()

for i in tqdm(range(len(train_data))):
    text, _ = train_data[i]
    word_counter.update(getPreprocessed_text(text).split())

vocab_size = len(word_counter)
embedding_dim = 100
hidden_size = 128
num_classes = len(labels)

print(vocab_size, num_classes)

  5%|▍         | 1686/33759 [00:14<04:40, 114.17it/s]


KeyboardInterrupt: 

In [15]:
# Mô hình phân loại văn bản
class TextClassifier(nn.Module):
    def __init__(self, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = output[:, -1, :]
        logits = self.fc(output)
        return logits

In [16]:
# Cấu hình
batch_size = 32
epochs = 10
learning_rate = 0.001

# Chuẩn bị dữ liệu
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Khởi tạo mô hình và tối ưu hóa
model = TextClassifier(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# Huấn luyện
model.train()
for epoch in tqdm(range(epochs)):
    for inputs, labels in train_loader:
        print(inputs)
        print(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: new(): invalid data type 'str'

In [ ]:
# Đánh giá mô hình
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy}%")

#### Xây dựng tập dữ liệu huấn luyện và kiểm thử

#### Xây dựng mô hình phân loại

### Đánh giá mô hình phân loại văn bản

### Tài liệu tham khảo